In [526]:
import numpy as np    

In [527]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [528]:
import nltk, string

# nltk.download('punkt') # if necessary...

In [529]:
from stop_words import get_stop_words

## Carga documentos del CORPUS

In [530]:
document = open('CORPUS.csv','r')
corpus = [doc.replace(';','').replace('\n','') for doc in document]

In [531]:
#print(documents.read())
print(corpus)

['Ley 22/2011, de 28 de julio, de residuos y suelos contaminados.', 'Ley 22/2011, de 28 de julio, de residuos y suelos no contaminados.', 'Orden PRA/321/2017, de 7 de abril, por la que se regulan los procedimientos de determinación de las emisiones de los contaminantes atmosféricos SO2, NOx, partículas y CO procedentes de las grandes instalaciones de combustión, el control de los instrumentos de medida y el tratamiento y remisión de la información relativa a dichas emisiones.', 'Real Decreto 198/2017, de 3 de marzo, por el que se modifican el Real Decreto 1728/2007, de 21 de diciembre, por el que se establece la normativa básica de control que deben cumplir los operadores del sector lácteo y se modifica el Real Decreto 217/2004, de 6 de febrero, por el que se regulan la identificación y registro de los agentes, establecimientos y contenedores que intervienen en el sector lácteo, y el registro de los movimientos de la leche, el Real Decreto 752/2011, de 27 de mayo, por el que se estable

## Preprocesado

### Stop Words, Stemmization, Tokenization, LowerCase, and Remove Punctuation

In [532]:
additional_stop_words = ['https','-','amp','&amp;']
stop_words = get_stop_words('english') \
    + get_stop_words('spanish')  \
    + additional_stop_words

In [533]:
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

In [534]:
def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words= stop_words )

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

## Procesado

In [535]:
tfidf = vectorizer.fit_transform(corpus)

In [536]:
pairwise_similarity = (tfidf * tfidf.T)
arr = pairwise_similarity.A

In [537]:
pairwise_similarity

<34x34 sparse matrix of type '<class 'numpy.float64'>'
	with 824 stored elements in Compressed Sparse Row format>

In [538]:
arr

array([[1.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.05142435],
       [1.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.05142435],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.08361992],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        0.        ],
       [0.05142435, 0.05142435, 0.08361992, ..., 0.        , 0.        ,
        1.        ]])

In [539]:
### elimina la coincidencia de un doc consigo mismo      
np.fill_diagonal(arr, np.nan)                                                                                              

In [540]:
### Busca índice del doc del que vamos a buscar otro similar

In [541]:
input_doc = "Ley 22/2011, de 28 de julio, de residuos y suelos contaminados."          

input_idx = corpus.index(input_doc)    

### Imprime el índice del doc del que vamos a buscar otro similar

In [542]:
input_idx

0

### Encuentra el índice del doc con mayor similaridad

In [543]:
result_idx = np.nanargmax(arr[input_idx]) 

In [544]:
result_idx

1

### Coeficiente de similaridad

In [545]:
arr[input_idx,result_idx]

1.0000000000000002

### Imprime el doc con el coeficiente de mayor similaridad

In [546]:
  corpus[result_idx]    

'Ley 22/2011, de 28 de julio, de residuos y suelos no contaminados.'

In [547]:
### Obtiene los índices de los 5 docs que más similitud tienen con el de entrada

In [548]:
idx = (-arr[input_idx]).argsort()[:5]

In [549]:
idx

array([ 1, 30,  8, 24, 20])

In [550]:
### Obtiene los coeficientes de similaridad de los 5 docs más similares

In [551]:
[ arr[input_idx, index] for index in idx ]

[1.0000000000000002,
 0.2815521658760466,
 0.1751969929483945,
 0.16665159044312214,
 0.10270794321858764]

In [552]:
### Lista los 5 docs con mayor similaridad al de entrada

In [553]:
[corpus[index] for index in idx]

['Ley 22/2011, de 28 de julio, de residuos y suelos no contaminados.',
 'Real Decreto 9/2005, de 14 de enero, por el que se establece la relación de actividades potencialmente contaminantes del suelo y los criterios y estándares para la declaración de suelos contaminados.',
 'Ley 21/2015, de 20 de julio, por la que se modifica la Ley 43/2003, de 21 de noviembre, de Montes.',
 'Ley 4/2012, de 28 de diciembre, de Medidas Financieras y Administrativas de la Comunidad Autónoma de Extremadura.',
 'Real Decreto 815/2013, de 18 de octubre, por el que se aprueba el Reglamento de emisiones industriales y de desarrollo de la Ley 16/2002, de 1 de julio, de prevención y control integrados de la contaminación.']